In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

import datasetclass
from models import network


In [3]:
'''
Codes Below were used to do a final preprocessing and save the final ready data in final_ready
'''

'\nCodes Below were used to do a final preprocessing and save the final ready data in final_ready\n'

```python
df = pd.read_csv("./preprocessing/final_data.csv", sep=';').drop(['Unnamed: 0', 'alias', 'descriptionProcessed'], axis=1)
encoder = OneHotEncoder()
weekdays = df['weekday'].values.reshape(-1, 1)
encoder.fit(weekdays)
one_hot_encoded = encoder.transform(weekdays).toarray().astype(np.int8)
df.drop(['weekday'], axis=1, inplace=True)
encoded_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(['weekday']))
encoded_df = pd.concat([df, encoded_df], axis=1)
encoded_df['multipleImage'] = encoded_df['multipleImage'].astype(np.int8)

print(len(encoded_df.keys()))
encoded_df.describe()
```

```python
path = './preprocessing/final_data_ready.csv'
encoded_df.to_csv(path = './preprocessing/final_data_ready.csv')
```

# Training

In [4]:
path = './preprocessing/final_data_ready.csv'
device = ('cuda' if torch.cuda.is_available() else 'cpu')

dataset_train = datasetclass.InstagramUserData(path, device, train=True)
dataset_test = datasetclass.InstagramUserData(path, device, train=False)
dataset_train.__len__(), dataset_test.__len__()

(5682, 632)

In [5]:
# Create Dataloader
train_loader = DataLoader(dataset_train, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=True)
# Test Dataloader
some_data, some_label = next(iter(train_loader))
in_dim = some_data.shape[-1]
some_data[0].shape, some_label[1].shape 

(torch.Size([1, 514]), torch.Size([1, 1]))

In [6]:
print(f"some data is like: {some_data[0]}\nand some corresponding label is like: {some_label[0]}")

some data is like: tensor([[0.0000e+00, 0.0000e+00, 2.6536e+03, 1.6680e+03, 1.9555e+05, 2.3000e+02,
         1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
         

In [7]:
# Load the neural network
model = network.LikeCategoryPredictor(in_dim=in_dim)
print(model)

LikeCategoryPredictor(
  (feature): Sequential(
    (0): Conv1d(514, 256, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv1d(256, 128, kernel_size=(1,), stride=(1,))
    (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv1d(128, 64, kernel_size=(1,), stride=(1,))
    (7): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv1d(64, 32, kernel_size=(1,), stride=(1,))
  )
  (predictor): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=32, out_features=10, bias=True)
    (2): Softmax(dim=10)
  )
)


In [ ]:
# First Check if everything is compatible then:
# Training